# DonorChoose.org 

### computition https://www.kaggle.com/c/donorschoose-application-screening

### reference https://www.kaggle.com/ahmedalesh/glove-xgboost-and-lgbm/code

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from string import punctuation
from tqdm import tqdm_notebook
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
from lightgbm import LGBMClassifier
from string import digits
from nltk.stem import PorterStemmer
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, roc_auc_score
import os

In [3]:
#READ TRAINING, TESTING AND RESOURCES FILES
train = pd.read_csv('Data/train.csv', delimiter=',')
test = pd.read_csv('Data/test.csv', delimiter=',')
res = pd.read_csv('Data/resources.csv', delimiter=',')

C:\Users\DELL\Anaconda3\envs\studypd35\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [5]:
test.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects
0,p233245,5724a0c3ce11008366fff36dab4b943c,Ms.,CA,2016-04-27 13:45:41,Grades PreK-2,Music & The Arts,Visual Arts,Art Will Make You Happy!,My 2nd grade students are amazing! They are v...,My class is made up of 12 boys and 12 girls. ...,My second grade class will really benefit from...,The genorous donations to my project will make...,My students need a drying rack for their art p...,2
1,p096795,445619941dc7cbe81c7be109dc61a56a,Mrs.,SC,2016-04-28 12:43:56,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Keeping up with the TIMEs,Students within the classroom work in small gr...,My students are all very talented young indivi...,"We do a lot of small group, where the student ...",This project is very important to my classroom...,My students need Time Magazines for next year ...,1
2,p236235,e92a4902b1611a189643d6f12c51e6a0,Mrs.,SC,2016-04-29 21:16:05,Grades PreK-2,Math & Science,"Applied Sciences, Mathematics",Building Bridges to Problem Solving,My students share a love of learning. These s...,My class consists of 14 energetic learners. O...,These Fairy Tale Problem Solving STEM kits wil...,These materials will be help my students with ...,My students need to be mentally stimulated to ...,0
3,p233680,8e92622d2985d3faac1de71609c4be72,Mrs.,IA,2016-04-27 22:32:43,Grades PreK-2,Literacy & Language,Literacy,Classroom Library,Reading is the gateway to the soul. Guiding c...,First graders enter the classroom each day rea...,Book bins will help to organize our classroom ...,"When students begin the first grade, many are ...",My students need an organized classroom library.,0
4,p171879,91a3c89981f626d9a0d067c65fb186ce,Mr.,CA,2016-04-27 18:59:15,Grades 6-8,"Music & The Arts, Special Needs","Performing Arts, Special Needs",Reeds so we can Read,"\""Mr. Reyes! I need another reed!\"" I hear t...",We have a diverse population with almost entir...,Each day in my class students consume supplies...,My students come from very troubled homes and ...,My students need reeds to perform in class eac...,1


In [6]:
res.head(5)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [7]:
#OBTAIN TOTAL PRICE REQUESTED FOR: QUANTITY * PRICE
res['total_requested'] = res['quantity'] * res['price']

In [8]:
#OBTAIN MEAN, SUM, MAX, MIN, COUNT FOR EACH PROJECT ID FOR BOTH TEST AND TRAINING FILES
tmp = res.groupby('id', as_index=False)['total_requested'].sum().rename(columns={'total_requested':'sum_requested'})
train = pd.merge(train, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].mean().rename(columns={'total_requested':'mean_requested'})
train = pd.merge(train, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].max().rename(columns={'total_requested':'max_requested'})
train = pd.merge(train, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].min().rename(columns={'total_requested':'min_requested'})
train = pd.merge(train, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].count().rename(columns={'total_requested':'count_requested'})
train = pd.merge(train, tmp, on='id', how='left')

In [9]:
tmp = res.groupby('id', as_index=False)['total_requested'].sum().rename(columns={'total_requested':'sum_requested'})
test = pd.merge(test, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].mean().rename(columns={'total_requested':'mean_requested'})
test = pd.merge(test, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].max().rename(columns={'total_requested':'max_requested'})
test = pd.merge(test, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].min().rename(columns={'total_requested':'min_requested'})
test= pd.merge(test, tmp, on='id', how='left')
tmp = res.groupby('id', as_index=False)['total_requested'].count().rename(columns={'total_requested':'count_requested'})
test = pd.merge(test, tmp, on='id', how='left')

In [10]:
train.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,...,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,sum_requested,mean_requested,max_requested,min_requested,count_requested
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1,899.94,449.970,449.97,449.97,2
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,...,NaN,NaN,My students need matching shirts to wear for d...,1,0,400.00,400.000,400.00,400.00,1
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1,469.99,469.990,469.99,469.99,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,...,NaN,NaN,My students need balls and other activity equi...,16,0,684.47,136.894,354.99,18.95,5
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,...,NaN,NaN,My students need a water filtration system for...,42,1,711.00,711.000,711.00,711.00,1


In [11]:
#OBTAIN OTHER FEATURES RELEVANT TO DATE CORRESPONDING TO DAY, MONTH, YEAR, HOUR, MINUTE AND SECOND FOR TRAINING AND TESTING SET
train['project_submitted_datetime'] = pd.to_datetime(train['project_submitted_datetime'], format='%Y-%m-%d %H:%M:%S')
train['day'] = train['project_submitted_datetime'].dt.day
train['month'] = train['project_submitted_datetime'].dt.month
train['year'] = train['project_submitted_datetime'].dt.year
train['hour'] = train['project_submitted_datetime'].dt.hour
train['minute'] = train['project_submitted_datetime'].dt.minute
train['second'] = train['project_submitted_datetime'].dt.second

In [12]:
test['project_submitted_datetime'] = pd.to_datetime(test['project_submitted_datetime'], format='%Y-%m-%d %H:%M:%S')
test['day'] = test['project_submitted_datetime'].dt.day
test['month'] = test['project_submitted_datetime'].dt.month
test['year'] = test['project_submitted_datetime'].dt.year
test['hour'] = test['project_submitted_datetime'].dt.hour
test['minute'] = test['project_submitted_datetime'].dt.minute
test['second'] = test['project_submitted_datetime'].dt.second

In [13]:
#FILL MISSING PREFIXES WITH TEACHER AND GROUP 'Ms' and 'Mrs.' into a single class and others in a different class
train['teacher_prefix'].fillna('Teacher', inplace=True)
test['teacher_prefix'].fillna('Teacher', inplace=True)
def teacher_prefix(s):
    if s in ['Ms.', 'Mrs.']:
        return 0
    else:
        return 1
train['teacher_prefix'] = train['teacher_prefix'].map(teacher_prefix)
test['teacher_prefix'] = test['teacher_prefix'].map(teacher_prefix)

In [14]:
#GROUP TEACHER ID BASED ON TEACHER NUMBER OF PREVIOUSLY POSTED PROJECTS CORRESPONDING TO THE SUM, MEAN, MAX, AND MIN 
tmp = train.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].sum().rename(columns={'teacher_number_of_previously_posted_projects':'sum_previous'})
train = pd.merge(train, tmp, on='teacher_id', how='left')
tmp = train.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].mean().rename(columns={'teacher_number_of_previously_posted_projects':'mean_previous'})
train = pd.merge(train, tmp, on='teacher_id', how='left')
tmp = train.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].max().rename(columns={'teacher_number_of_previously_posted_projects':'max_previous'})
train = pd.merge(train, tmp, on='teacher_id', how='left')
tmp = train.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].min().rename(columns={'teacher_number_of_previously_posted_projects':'min_previous'})
train = pd.merge(train, tmp, on='teacher_id', how='left')

tmp = test.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].sum().rename(columns={'teacher_number_of_previously_posted_projects':'sum_previous'})
test = pd.merge(test, tmp, on='teacher_id', how='left')
tmp = test.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].mean().rename(columns={'teacher_number_of_previously_posted_projects':'mean_previous'})
test = pd.merge(test, tmp, on='teacher_id', how='left')
tmp = test.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].max().rename(columns={'teacher_number_of_previously_posted_projects':'max_previous'})
test = pd.merge(test, tmp, on='teacher_id', how='left')
tmp =test.groupby('teacher_id', as_index=False)['teacher_number_of_previously_posted_projects'].min().rename(columns={'teacher_number_of_previously_posted_projects':'min_previous'})
test = pd.merge(test, tmp, on='teacher_id', how='left')

In [15]:
#LABEL ENCODE TEACHER_ID, SCHOOL STATE, PROJECT GRADE CATEGORY, PROJECT SUBJECT CATEOGRIES, AND PROJECT SUBJECT SUB CATEGORIES 
columns = ['teacher_id', 'school_state', 'project_grade_category', 'project_subject_categories', 'project_subject_subcategories']
for col in tqdm(columns):
    le = LabelEncoder()
    le = le.fit(pd.concat([train[col], test[col]], axis=0))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

100%|██████████| 5/5 [00:04<00:00,  1.13it/s]


In [17]:
#SELECT REMAINING COLUMNS IN TEXT AND FILL MISSING VALUES WITH EMPTY SPACE
remaining_columns = [col for col in train.columns if train[col].dtype =='object' and col !='id']
for col in remaining_columns:
    train[col].fillna(' ',inplace=True)
    test[col].fillna(' ', inplace=True)

    
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def clean_sentences(text):
    #Input: Sentences
    #Output: cleaned sentence by removing numbers, stopwords, punctuations, single alphabet, and change single uppercase to lowercase letters
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)    
    text = tokenizer.tokenize(text)
    text = [s.lower() for s in text]
    text = [s for s in text if s not in set(stopwords.words('english'))]
    text = [s for s in text if len(s)>1]
    text = ' '.join(text)
    return text   

In [34]:
def read_glove_vecs(glove_file):
    #input: file
    #output: word to 50d vector mapping output
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return word_to_vec_map
word_to_vec_map = read_glove_vecs('Data/glove50d/glove.6B.50d.txt')

In [35]:
def prepare_sequence(ds, word_to_vec_map):
    #input: Series, and word_to_vec_map of size(vocab_size,50)
    #output: returns shape of (len(ds), 50)
    traintest_X = []
    for sentence in tqdm(ds.values):
        sequence_words = np.zeros((word_to_vec_map['cucumber'].shape))
        for word in sentence.split():
            if word in word_to_vec_map.keys():
                temp_X = word_to_vec_map[word]
            else:
                temp_X = word_to_vec_map['#']
            sequence_words+=(temp_X)/len(sentence)
        traintest_X.append(sequence_words)
    return np.array(traintest_X)

In [36]:
#concatenate all sequences for training and testing set
train_w2v = prepare_sequence(train[remaining_columns[0]], word_to_vec_map)
test_w2v = prepare_sequence(test[remaining_columns[0]], word_to_vec_map)
for col in remaining_columns[1:]:
    temp_train = prepare_sequence(train[col], word_to_vec_map)
    temp_test = prepare_sequence(test[col], word_to_vec_map)
    train_w2v = np.concatenate([train_w2v, temp_train], axis=-1)
    test_w2v = np.concatenate([test_w2v, temp_test], axis=-1)

100%|██████████| 78035/78035 [00:06<00:00, 11669.66it/s]


In [37]:
#choose columns excluding target and datetime features
selected_columns = [col for col in train.columns if train[col].dtype != 'object' and col not in ['project_is_approved', 'project_submitted_datetime']]

In [38]:
#prepare trainig and testing set model input
train_X = np.concatenate([train[selected_columns].values, train_w2v], axis=-1)
train_Y = train['project_is_approved'].values
test_X = np.concatenate([test[selected_columns].values, test_w2v], axis=-1)

print('train_features: ', train_X.shape)
print('test_features: ', test_X.shape)

train_features:  (182080, 322)
test_features:  (78035, 322)


In [39]:
#shuffle data
random_id = np.random.permutation(len(train_X))
train_X = train_X[random_id]
train_Y = train_Y[random_id]

In [40]:
#train test split
tra_X, val_X, tra_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=1)
print('train_features: ', tra_X.shape)
print('validation_features: ', val_X.shape)
print('train_target: ', tra_Y.shape)
print('validation_target: ', val_Y.shape)

train_features:  (127456, 322)
validation_features:  (54624, 322)
train_target:  (127456,)
validation_target:  (54624,)


In [41]:
#parameters for xgboost and lgbm
XGB_params = {'num_round':200}
xgb = XGBClassifier(max_depth = 6, learning_rate=0.1, estimator =100, **XGB_params)
xgb.fit(tra_X, tra_Y, eval_set= [(tra_X, tra_Y), (val_X, val_Y)], eval_metric = 'auc', verbose=True)

lgb= LGBMClassifier(num_leaves =100, max_depth = 6, n_estimators =100)
lgb.fit(tra_X, tra_Y, eval_set= [(tra_X, tra_Y), (val_X, val_Y)], eval_metric = 'auc',verbose=True)



XGB_params = {'num_round':200}
xgb = XGBClassifier(max_depth = 6, learning_rate=0.2, estimator =100, **XGB_params)
lgb= LGBMClassifier(num_leaves =100, max_depth = 6, n_estimators =100)
xgb.fit(train_X, train_Y, eval_set= [(train_X, train_Y)], eval_metric = 'auc', verbose=True)
lgb.fit(train_X, train_Y, eval_set= [(train_X, train_Y)], eval_metric = 'auc',verbose=True)

test_pred_xgb = xgb.predict_proba(test_X)
test_pred_lgb =lgb.predict_proba(test_X)

[0]	validation_0-auc:0.712961	validation_1-auc:0.697888
[1]	validation_0-auc:0.723961	validation_1-auc:0.706731
[2]	validation_0-auc:0.728368	validation_1-auc:0.710384
[3]	validation_0-auc:0.731861	validation_1-auc:0.711314
[4]	validation_0-auc:0.73473	validation_1-auc:0.713387
[5]	validation_0-auc:0.737649	validation_1-auc:0.714922
[6]	validation_0-auc:0.740588	validation_1-auc:0.717007
[7]	validation_0-auc:0.741903	validation_1-auc:0.717735
[8]	validation_0-auc:0.743573	validation_1-auc:0.718393
[9]	validation_0-auc:0.744916	validation_1-auc:0.718577
[10]	validation_0-auc:0.746381	validation_1-auc:0.719287
[11]	validation_0-auc:0.748094	validation_1-auc:0.720473
[12]	validation_0-auc:0.750098	validation_1-auc:0.72134
[13]	validation_0-auc:0.751697	validation_1-auc:0.721955
[14]	validation_0-auc:0.753267	validation_1-auc:0.722927
[15]	validation_0-auc:0.754546	validation_1-auc:0.723547
[16]	validation_0-auc:0.756056	validation_1-auc:0.72438
[17]	validation_0-auc:0.75776	validation_1-a

In [42]:
#ENSEMBLING
test_pred = (0.5 *test_pred_xgb + 0.5*test_pred_lgb)
df = pd.DataFrame({'id':test.id})
df['project_is_approved'] = test_pred[:,1]
df.to_csv('submission_ensembling.csv', index=False)